In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(
    "ignore",
    message="A value is trying to be set on a copy of a DataFrame",
    category=FutureWarning
)


In [2]:
# Load the notebook
records=pd.read_csv("data/InterviewTask/AI/records.csv")
gold_cases=pd.read_csv("data/InterviewTask/AI/gold_cases.csv")

In [3]:
len(records)

20000

## Claim Value Bande Based on Client Segment

In [4]:
gold_cases_p0=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P0"].values


In [5]:
records.loc[records['case_id'].isin( gold_cases['case_id'])].to_csv("data/InterviewTask/AI/gold_records.csv",index=False)

In [6]:
gold_cases_p1=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P1"].values

In [7]:
# records.loc[records['case_id'].isin( gold_cases_p1)]

In [8]:
gold_cases_p2=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P2"].values

In [9]:
# records.loc[records['case_id'].isin( gold_cases_p2)]


In [10]:
gold_cases_p3=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P3"].values

In [11]:
# records.loc[records['case_id'].isin( gold_cases_p3)]

# Process the data

In [12]:
records.columns

Index(['case_id', 'received_at', 'client_segment', 'jurisdiction',
       'service_line', 'claim_value_band', 'attachments_present',
       'free_text_summary', 'handler_notes', 'historical_outcome'],
      dtype='object')

In [13]:
#take the first 2000 rows
records_copy=records.loc[0:1500].copy()

#Drop columns ['received_at','free_text_summary', 'handler_notes', 'historical_outcome']
records_copy.drop(['received_at','free_text_summary', 'handler_notes'],axis=1,inplace=True)

#Replace null service_line with no data available
records_copy['service_line'].fillna("No data Available", inplace=True)

#Replace null client segment with no data available
records_copy['client_segment'].fillna("No data Available", inplace=True)

#Drop the cases without any case ids
records_copy.drop(records_copy.loc[records_copy["case_id"].isna()].index,axis=0,inplace=True)

#Replace null Jurisdiction with no data available
records_copy['jurisdiction'].fillna("No data Available", inplace=True)

#Replace the Null values in claim_value_band to Unknown
records_copy['claim_value_band'].fillna("Unknown", inplace=True)

#Replace Null values in the Attachment Present to False
records_copy['attachments_present'].fillna(False, inplace=True)

#Replace Null values in the historical_outcome to Unknown
records_copy['historical_outcome'].fillna("Unknown", inplace=True)

def covert_to_yesno(value):
    if value:
        return ("yes")
    else:
        return ("No")
#Convert Attachment Present to Yes No 
records_copy['attachments_present']=records_copy['attachments_present'].apply(covert_to_yesno)


In [14]:
import os
COLUMNS = ['case_id','severe_legal_or_regulatory_risk','business_critical_impact','potential_fraud','conflicting_information','complex_incident_details','policy_interpretation_issues','legal_disputes','jurisdictional_complexity','coverage_terms_unclear','exclusions_may_apply','new_or_unusual_claim_type','no_legal_or_fraud_concerns','unclear_incident_description','claim_invalid_or_fraudulent','required_conditions_not_met','risk_summary']


df_all_signals = pd.DataFrame(columns=COLUMNS)

for i in range(1,4):
    df=pd.read_csv(f"data/llm_out{i}.0.csv")
    df_all_signals = pd.concat( [df_all_signals, df],ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/llm_out1.0.csv'

In [ ]:
#merge the two dataframes
processed_df=pd.merge(records_copy, df_all_signals, on="case_id", how="left")

In [ ]:
from src.triage.predict import ClaimTriageEvaluator
claim_calculator=ClaimTriageEvaluator()

In [ ]:
row=processed_df.loc[0].to_dict()
risk_score=claim_calculator.calculate_risk_score(row)
priority=claim_calculator.determine_priority(risk_score)
action=claim_calculator.determine_action(row,risk_score)
confidence=claim_calculator.calculate_confidence(row)
extracted_signals=claim_calculator.build_extracted_signals(row)

In [ ]:
[risk_score,priority,action,confidence,extracted_signals]

[0.64,
 'P1',
 'Immediate escalation',
 75,
 '{"jurisdiction": "UK", "service_line": "Legal", "claim_value_band": "<50k", "legal_risk": "Yes", "fraud_risk": "No"}']

In [ ]:
processed_df.loc[0]

case_id                                                                      C-00001
client_segment                                                                   SMB
jurisdiction                                                                      UK
service_line                                                                   Legal
claim_value_band                                                                <50k
attachments_present                                                              yes
historical_outcome                                                           Settled
severe_legal_or_regulatory_risk                                                  Yes
business_critical_impact                                                          No
potential_fraud                                                                   No
conflicting_information                                                           No
complex_incident_details                                         

In [ ]:
df_all_signals.columns

Index(['case_id', 'severe_legal_or_regulatory_risk',
       'business_critical_impact', 'potential_fraud',
       'conflicting_information', 'complex_incident_details',
       'policy_interpretation_issues', 'legal_disputes',
       'jurisdictional_complexity', 'coverage_terms_unclear',
       'exclusions_may_apply', 'new_or_unusual_claim_type',
       'no_legal_or_fraud_concerns', 'unclear_incident_description',
       'claim_invalid_or_fraudulent', 'required_conditions_not_met',
       'risk_summary'],
      dtype='object')

In [1]:
from src.triage.validate import evaluation
evaluation("data/InterviewTask/AI/gold_cases.csv")

gold dataset and prediction dataset merged successfully


gold dataset and prediction dataset merged successfully


In [ ]:
gold_cases=pd.read_csv("data/InterviewTask/AI/gold_cases.csv")
gold_records=pd.read_csv("data/processeddata/processeddf.csv")

In [ ]:
merged_df = pd.merge(
    gold_cases,
    gold_records,
    on="case_id",
    how="inner"
)

In [ ]:
merged_df.to_csv("data/InterviewTask/AI/gold_records_merged.csv")

In [ ]:
# from src.triage.injest import preprocess_getstructrureddata
# preprocess_getstructrureddata("data/InterviewTask/AI/gold_records.csv")

In [ ]:
from src.triage.features import process_features
process_features()

1
The llm outputs are merged to a single dataframe
preprocessed dataset and the llm output merged successfully
Processed data saved to local Success fully


In [ ]:
merged_df.loc[merged_df['expected_priority']=='P0']

,case_id,expected_priority,expected_action,notes,client_segment,jurisdiction,service_line,claim_value_band,attachments_present,historical_outcome,...,new_or_unusual_claim_type,unclear_incident_description,claim_invalid_or_fraudulent,required_conditions_not_met,has_regulator_involvement,has_cross_border_elements,has_time_sensitivity,has_missing_documentation,mentions_fraud_or_arson,risk_summary
0,C-03470,P0,Immediate escalation,synthetic gold; risk=0.82,Mid-Market,UK,Advisory,250k-1m,No,Accepted,...,No,No,No,No,Yes,Yes,Yes,No,No,The claim involves significant regulatory risk...
1,C-19557,P0,Immediate escalation,synthetic gold; risk=1.00,Enterprise,EU,Insurance,>1m,yes,Accepted,...,No,No,No,No,No,No,No,No,No,The claim involves significant business interr...
2,C-03560,P0,Immediate escalation,synthetic gold; risk=0.80,Enterprise,UK,Advisory,250k-1m,yes,Unknown,...,No,No,No,No data available,No,Yes,No data available,No,No,The claim involves a hurricane scenario with o...
3,C-04657,P0,Immediate escalation,synthetic gold; risk=0.85,Enterprise,UK,Advisory,>1m,No,Escalated,...,No,Yes,No,Yes,No,No,No,Yes,No,The claim indicates unclear scope and missing ...
4,C-00796,P0,Immediate escalation,synthetic gold; risk=0.94,Mid-Market,UK,Legal,>1m,yes,Unknown,...,No,No,No,No,No,No,Yes,No,No,The claim has escalated to a legal dispute wit...
5,C-09848,P0,Immediate escalation,synthetic gold; risk=0.86,SMB,US,Legal,>1m,yes,Unknown,...,No,No,No,No,No,Yes,No data available,No,Yes,The claim presents complex incident details wi...
6,C-01628,P0,Immediate escalation,synthetic gold; risk=0.85,Mid-Market,US,Legal,250k-1m,yes,Escalated,...,No,No,No,No,Yes,No,No,No,No,The claim involves a legal threat and has esca...
7,C-02183,P0,Immediate escalation,synthetic gold; risk=0.90,Enterprise,UK,Insurance,>1m,yes,Escalated,...,No,No,No,No,No,No,No,No,Yes,The claim indicates storm-related damage and t...
8,C-19227,P0,Immediate escalation,synthetic gold; risk=0.86,Mid-Market,UK,Legal,>1m,yes,Settled,...,No,No,No,No,No,No,Yes,No,No,The claim involves a potential breach of contr...
9,C-10731,P0,Immediate escalation,synthetic gold; risk=0.83,SMB,UK,Legal,250k-1m,yes,Escalated,...,No data available,No,No,No data available,No data available,Yes,No data available,No,No,The claim involves a disputed jurisdiction rel...


In [ ]:
## RF Model

import pandas as pd

df = pd.read_csv("data/InterviewTask/gold_records.csv")
